# Session 3 - Creating a custom settings file and running a harvest
-------------------------

The Data Harvester enables researchers with reusable workflows for automatic data extraction from a range of data sources including spatial-temporal processing into useable formats. User provided data is auto-completed with a suitable set of spatial- and temporal-aligned covariates as a ready-made dataset for machine learning and agriculture models. In addition, all requested data layer maps are automatically extracted and aligned for a specific region and time period. 

The main workflow of the Harvester is as follows: 
1) Options and user settings (e.g., data layer selections, spatial coverage, temporal constraints, i/o directory names) are defined by the user in the notebook settings menu or can be loaded with a settings yaml file (e.g., settings/settings_v0.2_saved.yaml). All settings are also saved in a yaml file for reusability.
2) The notebook imports settings and all Python modules that include functionality to download and extract data for each data source. After settings are read in, checked, and processed into valid data retrieval (API) queries, all selected data layers are sequentially downloaded and then processed into a clean dataframe table and co-registered raster maps. The entire workflow can be run either completely automatically or individually by selecting only certain process parts in the Notebook.

Additional data sources can be best added by writing the API handlers and extraction functionalities as separate Python module, which are then imported by the Notebook. Currently the following data sources are supported by the following modules:

- 'getdata_slga.py': Soil Data from Soil and Landscape Grid of Australia (SLGA)
- 'getdata_landscape': Landscape data from Soil and Landscape Grid of Australia (SLGA)
- 'getdata_silo.py': Climate Data from SILO
- 'getdata_dem.py: 'National Digital Elevation Model (DEM) 1 Second plus Slope and Apect calculation
- 'getdata_dea_nci.py: 'Digital Earth Australia's (DEA) Geoscience Earth Observations via NCI server
- 'getdata_dea.py: 'Digital Earth Australia's (DEA) Geoscience Earth Observations via Open Web Service server provided by DEA
- 'getdata_radiometric.py': Geoscience Australia National Geophysical Compilation Sub-collection Radiometrics
- 'getdata_ee.py': Google Earth Engine API integration handler


## Import libraries


In [2]:
# Load general python libraries
import os
import time
from datetime import datetime
from os.path import exists
from pathlib import Path
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# Load geodata_harvester modules/functions/packages
# See each python file for detailed options
from geodata_harvester.widgets import harvesterwidgets as hw
from geodata_harvester.utils import init_logtable, update_logtable
from geodata_harvester.arc2meter import calc_arc2meter
from geodata_harvester import (getdata_dea, getdata_dem,  getdata_landscape,
                               getdata_radiometric, getdata_silo, getdata_slga,
                            utils)  # getdata_ee
from eeharvest import harvester as eeharvester

## DIY settings configuration file

Let's start with loading all user settings and options as specified in the settings file. For this example we provide a template file `data/settings_session1.yaml`. You can use the default settings in this file. 
Or you may changed the file directly, or point to a new file.
Or override any of the defaults throughout this notebook.
This is the core piece of the Data Harvester that makes the data collection reproduceable. You could give the settings file to someone else and they will end up with the same data collections.

In [4]:
# Build your own settings file with interactive widgets
tab_nest, w_settings, names_settings, w_load = hw.gen_maintab()
display(tab_nest) 
time.sleep(2)

In [11]:
# For the automated people you can also run something like...
# load_settingsfilename = 'data/settings_session1.yaml'
# settings = hw.load_settings(load_settingsfilename)

Settings loaded:
----------------
settings.infile : data/example-site_llara.csv
settings.outpath : results/
settings.colname_lat : Lat
settings.colname_lng : Long
settings.target_bbox : [149.769345, -30.335861, 149.949173, -30.206271]
settings.target_res : 6.0
settings.date_min : 2022-10-01
settings.time_intervals : 4
settings.date_max : 2022-11-30
settings.time_buffer : 7
settings.target_sources:
   'DEM': ['DEM']
   'Landscape': ['Slope', 'Aspect', 'Relief_300m']
   'Radiometric': ['radmap2019_grid_dose_terr_awags_rad_2019', 'radmap2019_grid_dose_terr_filtered_awags_rad_2019']
   'SILO': {'daily_rain': 'mean', 'max_temp': 'median', 'min_temp': 'median', 'monthly_rain': 'sum'}
   'SLGA': {'Bulk_Density': ['0-5cm'], 'Clay': ['0-5cm']}
   'GEE': {'preprocess': {'collection': 'LANDSAT/LC09/C02/T1_L2', 'coords': None, 'date': datetime.date(2021, 1, 1), 'end_date': datetime.date(2021, 12, 31), 'buffer': None, 'bound': None, 'mask_clouds': True, 'reduce': 'median', 'spectral': 'NDVI'}, 'dow

## Setup dataset of interest

Here we are reading in the point locations for which we want to extract data. A custom bounding box for which to extract raster data can be set in the settings file. If no bounding box provided, rasters are extracted for the region given by the point location extent plus an additional padding of 0.05 deg in Lat/Long (see code below).

In [15]:
# Load in the dataset defining our location of interest as a geopandas dataframe
gdfpoints = gpd.read_file(settings.infile)

# Assing the data to well-named variables
lngs = gdfpoints[settings.colname_lng].astype(float)
lats = gdfpoints[settings.colname_lat].astype(float)

# Check the data looks reasonable
gdfpoints

,Lat,Long,geometry
0,-30.264663,149.85268,None
1,-30.265302,149.884838,None
2,-30.265302,149.884838,None
3,-30.278542,149.838791,None
4,-30.275437,149.830843,None
...,...,...,...
77,-30.268262,149.87615,None
78,-30.257031,149.880983,None
79,-30.258505,149.891118,None
80,-30.261989,149.884329,None


In [16]:
print(f'Info: Selected bounding box: {settings.target_bbox}')

# Estimate resolution in meters:
lat_center = (settings.target_bbox[1]+settings.target_bbox[3])/2
xres_meters, yres_meters = calc_arc2meter(settings.target_res, lat_center)
print(f'Info: {settings.target_res} arcsec resolution corresponds to {xres_meters:.1f}m x {yres_meters:.1f}m in x,y direction respectively (at Latitude: {lat_center:.2f}).')

Info: Selected bounding box: [149.769345, -30.335861, 149.949173, -30.206271]
Info: 6.0 arcsec resolution corresponds to 160.2m x 185.2m in x,y direction respectively (at Latitude: -30.27).


## Download and process data from API sources

From here we automatically download and process sequentially a range of data sources as specified in the settings file (see next subsections: SLGA, SILO, DEA, DEM). Note that you may retrieve info and parameter input options for any function easily by running a function/method with a preceeding '?', e.g:
```
?getdata_slga.get_slga_layers
?utils
```

In [18]:
# Initiate a dataframe for logging all data output names and layer titles.
# Note that the log table is later updated with update_logtable(), 
# which also instantly saves a copy of the table of the current status.
df_log = init_logtable()

NameError: name 'init_logtable' is not defined

### SLGA Download

Here we download all requested data layers from the Soil and Landscape Grid of Australia (SLGA) for the given bounding box. Note that for this example we select the top soil (0 - 5cm) only. Optionally other layers and depths including confidence intervals can be extracted as well; for more details and options see getdata_slga.py.

In [6]:
# We can set the input options for each function call, and additional parameters may be set
# too. Check the documentation of each function for full list of options.
depth_min, depth_max = getdata_slga.identifier2depthbounds(list(settings.target_sources['SLGA'].values())[0])
slga_layernames = list(settings.target_sources['SLGA'].keys())

fnames_out_slga = getdata_slga.get_slga_layers(
    slga_layernames, 
    settings.target_bbox, 
    settings.outpath, 
    depth_min = depth_min, 
    depth_max= depth_max, 
    get_ci = True)

⚑ SLGA_Bulk_Density_0-5cm.tif already exists, skipping download
⚑ SLGA_Bulk_Density_0-5cm_5percentile.tif already exists, skipping download
⚑ SLGA_Bulk_Density_0-5cm_95percentile.tif already exists, skipping download
⚑ SLGA_Clay_0-5cm.tif already exists, skipping download
⚑ SLGA_Clay_0-5cm_5percentile.tif already exists, skipping download
⚑ SLGA_Clay_0-5cm_95percentile.tif already exists, skipping download


In [7]:
# Add download info to log dataframe
df_log = update_logtable(
    df_log, 
    fnames_out_slga, 
    slga_layernames, 
    'SLGA', 
    settings, 
    layertitles = [], loginfos = 'downloaded')
df_log

,layername,agfunction,dataset,layertitle,filename_out,loginfo
0,Bulk_Density,0-5cm,SLGA,Bulk_Density_0-5cm,../dataresults/SLGA_Bulk_Density_0-5cm.tif,downloaded
1,Clay,0-5cm,SLGA,Clay_0-5cm,../dataresults/SLGA_Clay_0-5cm.tif,downloaded


### SILO Download

Here we download climate data layers from SILO and extract raster for the given bounding box and year.
For more details see getdata_silo.py

In [8]:
# Each data-source must be handled differently (as the data is stored in different ways)
# Here we must get each layer, one by one. The simplest way is to loop through them.
# Get data for each layer
outpath = settings.outpath+'_silo'
silo_layernames = list(settings.target_sources['SILO'].keys())
# run the download
fnames_out_silo = getdata_silo.get_SILO_layers(
    silo_layernames, 
    settings.date_min, 
    settings.date_max,
    outpath, 
    bbox = settings.target_bbox, 
    format_out = 'tif')

# Add download info to log dataframe
# TBD need to be tested for multiple years and not only one
if len(fnames_out_silo) > len(silo_layernames):
    # TBD Temporary solution for multiple years:
    nyears = int(len(fnames_out_silo)/len(silo_layernames))
    silo_layernames = silo_layernames * nyears
df_log = update_logtable(df_log, fnames_out_silo, silo_layernames, 'SILO', settings, layertitles = [], loginfos = 'downloaded')
df_log

⚑ daily_rain for 2022 already exists, skipping download
⚑ max_temp for 2022 already exists, skipping download
⚑ min_temp for 2022 already exists, skipping download
⚑ monthly_rain for 2022 already exists, skipping download


,layername,agfunction,dataset,layertitle,filename_out,loginfo
0,Bulk_Density,0-5cm,SLGA,Bulk_Density_0-5cm,../dataresults/SLGA_Bulk_Density_0-5cm.tif,downloaded
1,Clay,0-5cm,SLGA,Clay_0-5cm,../dataresults/SLGA_Clay_0-5cm.tif,downloaded
2,daily_rain,mean,SILO,daily_rain_mean,../dataresults/_silo/silo_daily_rain_2022-10-0...,downloaded
3,max_temp,median,SILO,max_temp_median,../dataresults/_silo/silo_max_temp_2022-10-01-...,downloaded
4,min_temp,median,SILO,min_temp_median,../dataresults/_silo/silo_min_temp_2022-10-01-...,downloaded
5,monthly_rain,sum,SILO,monthly_rain_sum,../dataresults/_silo/silo_monthly_rain_2022-10...,downloaded


### DEA Download

Here we download satellite data from Digital Earth Australia (DEA) within the given bounding box and for all available image capture dates that are available within the specified year(s). For more details see getdata_dea.py or getdata_dea_nci
.py

In [9]:
dea_layernames = settings.target_sources['DEA']

# These are multiple files, so we put them in a subdirectory to make subsequent processing easier.
outpath_dea = os.path.join(settings.outpath,'mvp_dea')

outfnames = getdata_dea.get_dea_layers_daterange(
    dea_layernames, 
    settings.date_min,
    settings.date_max,
    settings.target_bbox, 
    settings.target_res, 
    outpath_dea, 
    crs = 'EPSG:4326', 
    format_out = 'GeoTIFF')


⚑ landsat_barest_earth.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ ga_ls_ard_3.tif already exists, skipping download
⚑ g

#### DEA Processing

This aggregates all images for the given year(s) and gnerates a combined image, here for example for the mean and 5th and 95th percentile each.

In [10]:
layer_list = []
for layername in dea_layernames:
    s = sum(layername in s for s in outfnames)
    l = [layername]*s
    layer_list.append(l)

layer_list  = sum(layer_list, [])

layer_titles = [os.path.splitext(x)[0].split('/')[-1] for x in outfnames]

In [11]:
# Add extracted data info to log table
df_log = update_logtable(
    df_log, 
    outfnames, 
    layer_titles, 
    'DEA', 
    settings, 
    layertitles = layer_titles, 
    loginfos = 'processed',force=True)
#print(df_log.layertitle)
df_log

,layername,agfunction,dataset,layertitle,filename_out,loginfo
0,Bulk_Density,0-5cm,SLGA,Bulk_Density_0-5cm,../dataresults/SLGA_Bulk_Density_0-5cm.tif,downloaded
1,Clay,0-5cm,SLGA,Clay_0-5cm,../dataresults/SLGA_Clay_0-5cm.tif,downloaded
2,daily_rain,mean,SILO,daily_rain_mean,../dataresults/_silo/silo_daily_rain_2022-10-0...,downloaded
3,max_temp,median,SILO,max_temp_median,../dataresults/_silo/silo_max_temp_2022-10-01-...,downloaded
4,min_temp,median,SILO,min_temp_median,../dataresults/_silo/silo_min_temp_2022-10-01-...,downloaded
...,...,...,...,...,...,...
63,ga_ls_ard_3_2022-11-25,None,DEA,ga_ls_ard_3_2022-11-25,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-25.tif,processed
64,ga_ls_ard_3_2022-11-26,None,DEA,ga_ls_ard_3_2022-11-26,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-26.tif,processed
65,ga_ls_ard_3_2022-11-27,None,DEA,ga_ls_ard_3_2022-11-27,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-27.tif,processed
66,ga_ls_ard_3_2022-11-28,None,DEA,ga_ls_ard_3_2022-11-28,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-28.tif,processed


### DEM Download

Here we download and extract the National Digital Elevation Model (DEM), and also generate slope and aspect rasters from the extracted DEM. 
For more details see getdata_dem.py

In [12]:
outpath = os.path.join(settings.outpath, "mvp_dem")
dem_layernames = settings.target_sources['DEM']
outfnames = getdata_dem.get_dem_layers(dem_layernames, outpath, settings.target_bbox, settings.target_res)

# Add extracted data to log dataframe
df_log = update_logtable(
    df_log, 
    outfnames, 
    dem_layernames, 
    'DEM', 
    settings, 
    layertitles = dem_layernames,
    loginfos = 'downloaded')
df_log

⊙ Retrieving coverage from WCS server 3.6s                                                                     
⚑ DEM_SRTM_1_Second_Hydro_Enforced_2023_01_17.tif already exists, skipping download


,layername,agfunction,dataset,layertitle,filename_out,loginfo
0,Bulk_Density,0-5cm,SLGA,Bulk_Density_0-5cm,../dataresults/SLGA_Bulk_Density_0-5cm.tif,downloaded
1,Clay,0-5cm,SLGA,Clay_0-5cm,../dataresults/SLGA_Clay_0-5cm.tif,downloaded
2,daily_rain,mean,SILO,daily_rain_mean,../dataresults/_silo/silo_daily_rain_2022-10-0...,downloaded
3,max_temp,median,SILO,max_temp_median,../dataresults/_silo/silo_max_temp_2022-10-01-...,downloaded
4,min_temp,median,SILO,min_temp_median,../dataresults/_silo/silo_min_temp_2022-10-01-...,downloaded
...,...,...,...,...,...,...
64,ga_ls_ard_3_2022-11-26,None,DEA,ga_ls_ard_3_2022-11-26,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-26.tif,processed
65,ga_ls_ard_3_2022-11-27,None,DEA,ga_ls_ard_3_2022-11-27,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-27.tif,processed
66,ga_ls_ard_3_2022-11-28,None,DEA,ga_ls_ard_3_2022-11-28,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-28.tif,processed
67,ga_ls_ard_3_2022-11-29,None,DEA,ga_ls_ard_3_2022-11-29,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-29.tif,processed


### Landscape

Download landscape data from Soil and Landscape Grid of Australia (SLGA).

In [13]:
# Download landscape data
layernames = settings.target_sources['Landscape']
layertitles = ['Landscape_' + layername for layername in layernames]

outfnames = getdata_landscape.get_landscape_layers(
    layernames, 
    settings.target_bbox, 
    settings.outpath, 
    resolution = settings.target_res)

# Add extracted data to log dataframe
df_log = update_logtable(
    df_log, outfnames, 
    layernames, 
    'Landscape', 
    settings, 
    layertitles = layertitles,
    loginfos = 'downloaded')
df_log


⚑ Landscape_Slope.tif already exists, skipping download
⚑ Landscape_Aspect.tif already exists, skipping download
⚑ Landscape_Relief_300m.tif already exists, skipping download


,layername,agfunction,dataset,layertitle,filename_out,loginfo
0,Bulk_Density,0-5cm,SLGA,Bulk_Density_0-5cm,../dataresults/SLGA_Bulk_Density_0-5cm.tif,downloaded
1,Clay,0-5cm,SLGA,Clay_0-5cm,../dataresults/SLGA_Clay_0-5cm.tif,downloaded
2,daily_rain,mean,SILO,daily_rain_mean,../dataresults/_silo/silo_daily_rain_2022-10-0...,downloaded
3,max_temp,median,SILO,max_temp_median,../dataresults/_silo/silo_max_temp_2022-10-01-...,downloaded
4,min_temp,median,SILO,min_temp_median,../dataresults/_silo/silo_min_temp_2022-10-01-...,downloaded
...,...,...,...,...,...,...
67,ga_ls_ard_3_2022-11-29,None,DEA,ga_ls_ard_3_2022-11-29,../dataresults/mvp_dea/ga_ls_ard_3_2022-11-29.tif,processed
68,DEM,None,DEM,DEM,../dataresults/mvp_dem/DEM_SRTM_1_Second_Hydro...,downloaded
69,Slope,None,Landscape,Landscape_Slope,../dataresults/Landscape_Slope.tif,downloaded
70,Aspect,None,Landscape,Landscape_Aspect,../dataresults/Landscape_Aspect.tif,downloaded


### Radiometrics

Download maps of Geoscience Australia National Geophysical Compilation Sub-collection Radiometrics

In [14]:
# Download radiometrics
layernames = settings.target_sources['Radiometric']

outfnames = getdata_radiometric.get_radiometric_layers(
    settings.outpath, 
    layernames, 
    bbox = settings.target_bbox, 
    resolution=settings.target_res)

 # Add extracted data to log dataframe
df_log = update_logtable(
    df_log, outfnames, 
    layernames, 
    'Radiometric', 
    settings, 
    layertitles = layernames,
    loginfos = 'downloaded')
df_log

⚑ radmap2019_grid_dose_terr_awags_rad_2019.tif already exists, skipping download
⚑ radmap2019_grid_dose_terr_filtered_awags_rad_2019.tif already exists, skipping download


,layername,agfunction,dataset,layertitle,filename_out,loginfo
0,Bulk_Density,0-5cm,SLGA,Bulk_Density_0-5cm,../dataresults/SLGA_Bulk_Density_0-5cm.tif,downloaded
1,Clay,0-5cm,SLGA,Clay_0-5cm,../dataresults/SLGA_Clay_0-5cm.tif,downloaded
2,daily_rain,mean,SILO,daily_rain_mean,../dataresults/_silo/silo_daily_rain_2022-10-0...,downloaded
3,max_temp,median,SILO,max_temp_median,../dataresults/_silo/silo_max_temp_2022-10-01-...,downloaded
4,min_temp,median,SILO,min_temp_median,../dataresults/_silo/silo_min_temp_2022-10-01-...,downloaded
...,...,...,...,...,...,...
69,Slope,None,Landscape,Landscape_Slope,../dataresults/Landscape_Slope.tif,downloaded
70,Aspect,None,Landscape,Landscape_Aspect,../dataresults/Landscape_Aspect.tif,downloaded
71,Relief_300m,None,Landscape,Landscape_Relief_300m,../dataresults/Landscape_Relief_300m.tif,downloaded
72,radmap2019_grid_dose_terr_awags_rad_2019,None,Radiometric,radmap2019_grid_dose_terr_awags_rad_2019,../dataresults/radiometric_radmap2019_grid_dos...,downloaded


## Google Earth Engine

To connect to Google Earth Engine, you must already have access to the Google
Earth Engine API. You can request for access [by clicking
here](https://earthengine.google.com/signup/). Once you are authorised, connect
to the API using `initialise()`. A web browser may be invoked to complete the
process.

In [15]:
# Connect to the Google Earth Engine API
getdata_ee.initialise()

⊙ Initialising Earth Engine... 5.4s                                                                            
✔ Earth Engine authenticated


### Collect and preprocess Earth Engine Data

Use `collect()` to define the data object and `preprocess()` to perform server-side
data processing which includes cloud and shadow masking, image reduction and
calculation of spectral indices.

In [16]:
gee = settings.target_sources["GEE"]

# Define the collection, area of interest and date range
img = getdata_ee.collect(collection=gee['preprocess']['collection'],
              coords=settings.target_bbox,
              date=gee["preprocess"]["date"], 
              end_date = gee["preprocess"]["end_date"])

# Perform cloud maskng, reduction, and calculate one or more spectral indices
img.preprocess(mask_clouds=gee["preprocess"]["mask_clouds"], 
               reduce=gee["preprocess"]["reduce"], 
               spectral=gee["preprocess"]["spectral"])

ℹ Running preprocess()
⊙ Applying scale, offset and cloud masks... 0.3s                                                               
⊙ Computing spectral index: NDVI 0.6s                                                                          
⊙ Reducing image pixels by median 0.0s                                                                         
✔ Google Earth Engine preprocessing complete


### Download

Download the data using `download()` and add to `df_log`:

In [17]:
# Downoad data
img.download(bands=gee["download"]["bands"],
             scale=gee["download"]["scale"],
             outpath=settings.outpath,
             out_format=gee["download"]["format"])


# Add to log dataframe
outfnames = [settings.outpath + img.filenames]
layernames = [Path(img.filenames).resolve().stem]

df_log = update_logtable(
    df_log,
    outfnames,
    layernames,
    "GEE",
    settings,
    layertitles=[],
    agfunctions=img.reduce,
    loginfos="downloaded",
    )

df_log # preview

ℹ Running download()
ℹ Band(s) selected: NDVI
⚑ ee_LAN_20210101_20211231_NDVI_median_100m.tif already exists, skipping download
✔ Google Earth Engine download(s) complete


,layername,agfunction,dataset,layertitle,filename_out,loginfo
0,Bulk_Density,0-5cm,SLGA,Bulk_Density_0-5cm,../dataresults/SLGA_Bulk_Density_0-5cm.tif,downloaded
1,Clay,0-5cm,SLGA,Clay_0-5cm,../dataresults/SLGA_Clay_0-5cm.tif,downloaded
2,daily_rain,mean,SILO,daily_rain_mean,../dataresults/_silo/silo_daily_rain_2022-10-0...,downloaded
3,max_temp,median,SILO,max_temp_median,../dataresults/_silo/silo_max_temp_2022-10-01-...,downloaded
4,min_temp,median,SILO,min_temp_median,../dataresults/_silo/silo_min_temp_2022-10-01-...,downloaded
...,...,...,...,...,...,...
70,Aspect,None,Landscape,Landscape_Aspect,../dataresults/Landscape_Aspect.tif,downloaded
71,Relief_300m,None,Landscape,Landscape_Relief_300m,../dataresults/Landscape_Relief_300m.tif,downloaded
72,radmap2019_grid_dose_terr_awags_rad_2019,None,Radiometric,radmap2019_grid_dose_terr_awags_rad_2019,../dataresults/radiometric_radmap2019_grid_dos...,downloaded
73,radmap2019_grid_dose_terr_filtered_awags_rad_2019,None,Radiometric,radmap2019_grid_dose_terr_filtered_awags_rad_2019,../dataresults/radiometric_radmap2019_grid_dos...,downloaded


## Save the final log or start from here to re-load it in.
We have now completed the data download section. You may add additional downlods and processing steps to your log file.

In [ ]:
# Save out (or load in) the log file.
logfile = settings.outpath+'log.csv'
if exists(logfile):
    print(logfile, "exists! Do you want to read it in?\n")
    user_input = input("(y)es / (n)o / (a)bort ? Ansering 'no' will overwrite the current file.\n")
    if user_input=='y':
        df_log = pd.read_csv(settings.outpath+'log.csv')
    elif user_input =='n': 
        df_log.to_csv(settings.outpath+'log.csv',index=False)
        print(logfile, "saved!")
    else:
        print("Cancelling read/write for log file.\nFigure out what you want to do and please try again.")
else:
    print("No log file found. Saving to", settings.outpath+'log.csv')
    df_log.to_csv(settings.outpath+'log.csv',index=False)

df_log

../dataresults/log.csv exists! Do you want to read it in?



## Points extraction from downloaded/processed data

By default point values of all processed layers in df_log are extracted given by the input locations. However, you can select also only certain layers (see in code). 

In [ ]:
# Select all processed data
df_sel = df_log.copy()

# or select only the rasters of interest, for example:
"""
df_sel = df_log[df_log['layername'].isin(['DEM','Slope',
'landsat8_nbart_16day_channel0', 
'Organic_Carbon','Depth_of_Soil',
'mean_temp','monthly_rain'])]
"""

rasters= df_sel['filename_out'].values.tolist()
titles = df_sel['layertitle'].values.tolist()
    
# Extract datatable from rasters given input coordinates
gdf = utils.raster_query(lngs,lats,rasters,titles)

### Inspect result dataframe

In [ ]:
# Inspect either entire generated dataframe with 
# gdf
# or only the first rows
gdf.head()

In [ ]:
# Get some general info about result table:
gdf.info()

### Save the results table

Finally, the result dataframe table is saved as a csv file, which can be used now to do some awesome ML.
In addition the results are also saved as a geo-spatial referenced geopackage (.gpkg), which can be used again as input for further analysis or to inspect and overlay data on other layers and basemaps. The geopackage is a standard georeferenced file format and can be opened with any geo-spatial package or interactive software (e.g., QGIS, Esri ArcGIS). 

In [ ]:
# Save the results table to a csv 
gdf.to_csv(os.path.join(settings.outpath, "results.csv"), index = True, mode='w')

# Save also as geopackage
gdf.to_file(os.path.join(settings.outpath, "results.gpkg"), driver="GPKG")
# Note: The deprecated warning below is a bug in geopandas and will be fixed in their bext version.

### Overview plot of all processed rasters 

This provides a quick overview to inspect all processed data layers with an overlay of the requested location points.
  

In [ ]:
# Plot one of that datasets with the points on top
utils.plot_rasters(rasters,lngs,lats,titles)